@Abdulrahman Alabrash 

https://github.com/alabrashJr/DCNN-Julia


In [1]:
using Pkg;Pkg.update(); for p in ("Embeddings","DataFrames","DataStructures","DataFrames","FileIO","LinearAlgebra","Knet","FileIO"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using DataStructures,DataFrames,FileIO,Embeddings,LinearAlgebra,DataFrames
#using Knet: Knet, conv4, pool, mat, KnetArray, nll, zeroone, progress, sgd, param, param0, dropout, relu, Data,minibatch

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


Read data from saved file 
load("TREC_sib.jld2","datas")-> returns revs, W, W2, word_idx_map, vocab
* revs Dict{String,Any} with 5 entries:y,num_words,tree,text,split
    * y-> label of the questions 1-5 
    * num_words-> length of questions 
    * tree -> 46-element Array{Array,1}, each array contains data_sibling(5)+data_tree(5) =10
    * text -> the question text 
    * split -> type of tuple (training, test , div)



* W wordembedding generated by google2vec, size(10098×300)
* W2 word embedding generated by -0.25 , 0.25 size(10098×300)
* word_idx_map Dict{Any,Any} with 10097 entries , unique word id dictionary 
* vocab DefaultDict{Any,Any,Int64} with 10097 entries, vocab and reptation dictionary

In [22]:
labels=["abbreviation", "entity", "description", "location" ,"numeric"," "]
revs, W, W2, word_idx_map, vocab=load("Data/TREC_sib.jld2","datas"); 
word_idx_map["ROOT"] = 0;

In [23]:
#change W2 with prof's method 
struct Embed; w; end
Embed(vocabsize::Int,embedsize::Int) = Embed(param(embedsize,vocabsize))
(e::Embed)(x) = e.w[:,x]
W2=Embed(300,length(vocab))

UndefVarError: UndefVarError: param not defined

In [27]:
#Transforms sentence into a list of indices. Pad with zeroes.
function get_text_mat(t,word_idx_map;max_l=56,filter_h=5)
    #t the text of question
    x=[] # output matrix
    pad=filter_h -1 # padding number
    for i in collect(1:pad);push!(x,0);end #adding padding 
    words=split(t)
    #extract the unique id of words in the question text and adding it to the matrix 
    for w in words
        if w in keys(word_idx_map);push!(x,word_idx_map[w])
        else; @show w ;end
    end    

    while length(x)<max_l+2*pad    # accomplish 64 +1 size by adding zeros till finish 
            push!(x,0)
    end
    
    return  x
end

get_text_mat (generic function with 1 method)

In [28]:
#Transforms sentence into a list of indices. Pad with zeroes. 
function get_tree_rep(r,word_idx_map)
# question 
#@show t=r["tree"] #the tree of question
    each_sent=deepcopy(r)# output matrix
    for (j, each_word) in enumerate(each_sent[1:end-1])
        #@show (j, each_word)
            for (l, each_field) in enumerate(each_word)
           # @show (l, each_field)
                if each_field in keys( word_idx_map)
                #@show j,l ;
                    each_sent[j]=Array{Any,1}(each_sent[j])
                     each_sent[j][l] = word_idx_map[each_field]
                elseif each_field == 0
                    continue
                else
                    @show each_field
                end
            end
    end       
    return each_sent;
end

get_tree_rep (generic function with 1 method)

In [29]:
ran=rand(1:length(revs));@show ran
print(revs[ran]["text"]); println("\t",length(split(revs[ran]["text"])))
println(permutedims(get_text_mat(revs[ran]["text"],word_idx_map;max_l=56) ))
for i in split(revs[ran]["text"]) ;println(i,"=",word_idx_map[i]) ;end

ran = 518
How many years is Johnnie Walker Black Label aged ?	10
Any[0 0 0 0 2939 4443 9911 1189 3462 4824 8538 4481 8156 1557 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
How=2939
many=4443
years=9911
is=1189
Johnnie=3462
Walker=4824
Black=8538
Label=4481
aged=8156
?=1557


In [30]:
ran=1#rand(1:length(revs))
tree_ind=1
println(revs[ran]["tree"][tree_ind])
println(permutedims(get_tree_rep(revs[ran]["tree"],word_idx_map))[tree_ind])
for i in revs[ran]["tree"][tree_ind] ;println(i,"=",word_idx_map[i]) ;end

Any["ROOT", "ROOT", "ROOT", "ROOT", "How", "How", "develop", "*START*", "*START*", "ROOT"]
Any[0, 0, 0, 0, 2939, 2939, 8833, 8148, 8148, 0]
ROOT=0
ROOT=0
ROOT=0
ROOT=0
How=2939
How=2939
develop=8833
*START*=8148
*START*=8148
ROOT=0


In [9]:
function getSen(vector)
#labels=["abbreviation","numeric",  "description", "human","location" ,"entity"]
t=Array{Int}(vector)
println(permutedims(t))
for i in t
    if i==1557;print("?\n y =",t[end]+1 );break;end
    if i==0;continue;end
    for (key,value) in word_idx_map
        if value==i; print(key," ");end
    end
end
end

getSen (generic function with 1 method)

In [31]:
function train_dev_test(revs)
    s1,s2,s3=[],[],[]
    t1,t2,t3=[],[],[]
    for rev in revs
    sent =get_text_mat(rev["text"], word_idx_map)   
    push!(sent,rev["y"])
    sent_tensor = get_tree_rep(rev["tree"], word_idx_map)
        
    if rev["split"]==1
            push!(s1,Array{Int}(sent))
            push!(t1,sent_tensor)
    elseif rev["split"]==2
            push!(s2,Array{Int}(sent))
            push!(t2,sent_tensor)
    elseif rev["split"]==3
            push!(s3,Array{Int}(sent))
            push!(t3,sent_tensor)
    end
end

    train = hcat([f1 for f1 in s1]...)
    test =hcat([f1 for f1 in s2]...)
    dev = hcat([f1 for f1 in s3]...)
    train_tensor = t1
    test_tensor = t2
    dev_tensor = t3
    return (train,test,dev),(train_tensor,test_tensor,dev_tensor)
end
dataset,datasetTensor=train_dev_test(revs);


dataset[1] --> train 65×5452 } text data
dataset[2]----> test 65×500  }
dataset[3] ----> dev 0x0     }

datasetTensor[1] ---> train_tensor 5452(46(10)))
datasetTensor[2] ---> test_tensor 500(46(10)))
datasetTensor[3] --->dev_tensor 0(0(0))

In [10]:
#try on some vectors 
ran=rand(1:size(dataset[1],2))
getSen(dataset[1][:,ran])

[0 0 0 0 6965 2802 4039 1557 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2]
What are platelets ?
 y =3

In [317]:
y_train=[dataset[1][:,x][end] for x in 1:size(dataset[1],2)];
y_test=[dataset[2][:,x][end] for x in 1:size(dataset[2],2)];

In [318]:
dtrn=minibatch(dataset[1][1:end-1,:],y_train,160,shuffle=true) # 5451/170
dtst=minibatch(dataset[2][1:end-1,:],y_test,160)

Data{Tuple{Array{Int64,2},Array{Int64,1}}}([0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], [1 5 … 1 2], 160, 501, false, 342, 1:501, false, (64, 501), (501,), Array{Int64,2}, Array{Int64,1})

In [319]:
summary.(Iterators.first(dtrn))

("64×160 Array{Int64,2}", "160-element Array{Int64,1}")